# Dimensionality Reduction - Principal Component Analysis

Principal Component Analysis (PCA) is a dimensionality reduction technique that can be used to reduce dimensionality of a dataset while retaining most of the information in the dataset. This concept might be daunting at first, but it is actually quite simple. Let's start with a very simple example.

# Student scores

To understand PCA it's really a lot more fun if we use real data. We'll use a dataset of student scores on various tests that's available from [Kaggle](https://www.kaggle.com/datasets/spscientist/students-performance-in-exams). For now let's focused only on three features: math score, reading score, and writing score.


In [ ]:
import pandas as pd

# Replace 'your_data_file.csv' with the actual path to your CSV file
file_path = 'https://storage.googleapis.com/rg-ai-bootcamp/machine-learning/StudentsPerformance.csv'

# Load the CSV data into a pandas DataFrame
data = pd.read_csv(file_path)

numerical_data = data[['math score', 'reading score', 'writing score']]
numerical_data

,math score,reading score,writing score
0,72,72,74
1,69,90,88
2,90,95,93
3,47,57,44
4,76,78,75
...,...,...,...
995,88,99,95
996,62,55,55
997,59,71,65
998,68,78,77


Let's plot the data to see what it looks like.

In [ ]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create a subplot with axis labels
fig = make_subplots(rows=1, cols=3, subplot_titles=( "Reading vs Writing", "Math vs Reading", "Math vs Writing"))

# Reading vs Writing
fig.add_trace(go.Scatter(x=data['reading score'], y=data['writing score'], mode='markers', name='Reading vs Writing',
                         hovertemplate='Reading Score: %{x:.2f}<br>Writing Score: %{y:.2f}<extra></extra>'), row=1, col=1)
fig.update_xaxes(title_text="Reading Score", row=1, col=1)
fig.update_yaxes(title_text="Writing Score", row=1, col=1)

# Add scatter plots with axis labels
# Math vs Reading
fig.add_trace(go.Scatter(x=data['math score'], y=data['reading score'], mode='markers', name='Math vs Reading',
                         hovertemplate='Math Score: %{x:.2f}<br>Reading Score: %{y:.2f}<extra></extra>'), row=1, col=2)
fig.update_xaxes(title_text="Math Score", row=1, col=2)
fig.update_yaxes(title_text="Reading Score", row=1, col=2)

# Math vs Writing
fig.add_trace(go.Scatter(x=data['math score'], y=data['writing score'], mode='markers', name='Math vs Writing',
                         hovertemplate='Math Score: %{x:.2f}<br>Writing Score: %{y:.2f}<extra></extra>'), row=1, col=3)
fig.update_xaxes(title_text="Math Score", row=1, col=3)
fig.update_yaxes(title_text="Writing Score", row=1, col=3)

# Update layout
fig.update_layout(height=500, width=1500, title_text="Students' Performance Comparisons")
fig.show()


In [ ]:
numerical_data.corr()

,math score,reading score,writing score
math score,1.000000,0.817580,0.802642
reading score,0.817580,1.000000,0.954598
writing score,0.802642,0.954598,1.000000


In [ ]:
#corr -1 sampai 1
#cov -->

If we see above 1000 student scores on a plot, we can see some patterns:

- Students who perform well on reading tend to perform well on writing as well
- Students who perform well on reading or writing doesn't mean they perform well on math, and vice versa
- Students who do poor on any of the tests tend to do poor on all of the tests, and vice versa

As you can see the skills of reading and writing are more aligned than the skill of either of those with math. This "alignment" is something called **covariance**. Covariance is a measure of how two variables change together. Let's check below calculation.

In [ ]:
cov_reading_writing = data['reading score'].cov(data['writing score'])
print("Covariance between Reading Score and Writing Score:", cov_reading_writing)

cov_math_writing = data['math score'].cov(data['writing score'])
print("Covariance between Math Score and Writing Score:", cov_math_writing)

cov_math_reading = data['math score'].cov(data['reading score'])
print("Covariance between Math Score and Reading Score:", cov_math_reading)

Covariance between Reading Score and Writing Score: 211.78666066066063
Covariance between Math Score and Writing Score: 184.93913313313314
Covariance between Math Score and Reading Score: 180.99895795795803


As you can see above the math is validating our observation: reading and writing covariance score is higher than reading and math or writing and math. This means if a student either does well or poorly on reading, they tend to do the same on writing, so if we say have a class to improve student's reading skills, we can expect their writing skills to improve as well.

But because the covariance is still positive for math vs either reading or writing, can we say that if we improve student's reading or writing skills, their math skills will improve as well? Maybe, intuitively if a student is get a better score from reading or writing, they will become a better learner, but this conclusion is a long shot compared to the conclusion we can make about reading and writing where it's more obvious that if we improve one, the other will tend to improve as well.

In [ ]:
#feature selection
#simpel --> correlation (-1 strong negative sampai +1 strong positive) --> filter corr 0.5 keatas atau -0.5 kebawah
#

# Let's do some PCA

So before we already introduced our dataset and a little bit of covariance that will come in handy to understand PCA. Now let's do some PCA to our data. PCA is a technique that can be used to reduce the dimensionality of a dataset while trying to retain as much information as possible. So let's try to reduce our 3 features dataset into 2 features dataset.

In [ ]:
data_mnist = pd.read_csv(r'/content/sample_data/mnist_train_small.csv', header=None)

In [ ]:
data_mnist.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
numerical_data.head()

,math score,reading score,writing score
0,72,72,74
1,69,90,88
2,90,95,93
3,47,57,44
4,76,78,75


In [ ]:
#feature importances

In [ ]:
import plotly.express as px
from sklearn.decomposition import PCA

# Performing PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(numerical_data.drop(columns=['writing score']))

In [ ]:
numerical_data.head()

,math score,reading score,writing score
0,72,72,74
1,69,90,88
2,90,95,93
3,47,57,44
4,76,78,75


In [ ]:
pca.inverse_transform(principal_components)

array([[71.90864897, 73.59452708, 72.54809736],
       [69.07823603, 88.63439455, 89.24345719],
       [90.05460359, 94.04689748, 93.86785109],
       ...,
       [59.15688845, 68.26152096, 67.49353241],
       [68.0179996 , 77.68581799, 77.28607962],
       [76.99578252, 86.07361591, 85.93296876]])

In [ ]:
pca.explained_variance_ratio_

array([0.90893475, 0.09106525])

In [ ]:
sum(pca.explained_variance_ratio_)

0.9852328246233537

In [ ]:
pd.DataFrame

In [ ]:
#trade off ratio
#smeakin banyak mengurangi fitur --> smeakin cepat proses perhitungan tetapi banyak juga kehilangan informasi
#90% --> masih lama --> 80% --> 1 jam
#60% --> 15 menitan --> akurasinya sedikit berkurang

In [ ]:
#dengan mengurangi 1 features kita hanya kehilangan 1.2% informasi dari data
#dengan mengurangi 2 features kita hanya kehilangan 10% informasi dari data

In [ ]:
principal_components

array([[ -8.48837536,   1.26411978],
       [-25.46144129, -13.73117695],
       [-43.12175323,  -0.35950596],
       ...,
       [  4.75467372,  -5.15605377],
       [-11.46651782,  -5.47790938],
       [-26.47680822,  -4.83322812]])

In [ ]:
pca_df

,Principal Component 1,Principal Component 2,Math Score,Reading Score,Writing Score,Cluster
0,-8.488375,1.264120,72,72,74,0
1,-25.461441,-13.731177,69,90,88,3
2,-43.121753,-0.359506,90,95,93,3
3,32.036284,-0.878501,47,57,44,2
4,-14.777792,2.006871,76,78,75,0
...,...,...,...,...,...,...
995,-45.482281,-4.303784,88,99,95,3
996,18.199949,7.372887,62,55,55,2
997,4.754674,-5.156054,59,71,65,4
998,-11.466518,-5.477909,68,78,77,0


linear regesi --> decision tree --> random forest / knn / svm --> gradient boosting

In [ ]:
# Creating a DataFrame for the PCA results
pca_df = pd.DataFrame(data=principal_components, columns=['Principal Component 1', 'Principal Component 2'])
pca_df['Writing Score'] = data['writing score']

In [ ]:
pca_df

,Principal Component 1,Principal Component 2,Writing Score
0,-6.230231,2.034380,74
1,-16.490659,-13.056135,88
2,-35.132111,-2.170537,93
3,22.209973,-4.380820,44
4,-13.266710,0.457050,75
...,...,...,...
995,-36.447850,-6.444742,95
996,12.742102,7.424239,55
997,3.862811,-6.219741,65
998,-7.480572,-5.067497,77


In [ ]:
# Creating a DataFrame for the PCA results
pca_df = pd.DataFrame(data=principal_components, columns=['Principal Component 1', 'Principal Component 2'])

# Adding original scores as hover_data
pca_df['Math Score'] = data['math score']
pca_df['Reading Score'] = data['reading score']
pca_df['Writing Score'] = data['writing score']

# Create a scatter plot using Plotly
fig = px.scatter(
    pca_df,
    x='Principal Component 1',
    y='Principal Component 2',
    hover_data=['Math Score', 'Reading Score', 'Writing Score'],
    title='PCA of Student Performance',
    labels={'Principal Component 1': 'PC1', 'Principal Component 2': 'PC2'}
)

# Show the plot
fig.show()


In [ ]:
sum(pca.explained_variance_ratio_)

0.9852328246233539

In [ ]:
pca_df.corr()

,Principal Component 1,Principal Component 2,Math Score,Reading Score,Writing Score
Principal Component 1,1.000000e+00,-6.541003e-17,-0.916554,-0.971055,-0.967110
Principal Component 2,-6.541003e-17,1.000000e+00,0.399813,-0.177682,-0.212591
Math Score,-9.165545e-01,3.998129e-01,1.000000,0.817580,0.802642
Reading Score,-9.710551e-01,-1.776821e-01,0.817580,1.000000,0.954598
Writing Score,-9.671100e-01,-2.125912e-01,0.802642,0.954598,1.000000


Above is 2 dimensional plot of our data, reduced from 3 dimensions. The process of the data reduction is done by focusing on keeping the relation between features as much as possible. How so? Look at below plot

In [ ]:
import plotly.express as px
import plotly.subplots as sp


# Adding original scores as hover_data
pca_df['Math Score'] = data['math score']
pca_df['Reading Score'] = data['reading score']
pca_df['Writing Score'] = data['writing score']

# Create scatter plots using Plotly
fig1 = px.scatter(
    pca_df,
    x='Principal Component 1',
    y='Principal Component 2',
    hover_data=['Math Score', 'Reading Score', 'Writing Score'],
    title='PCA of Student Performance (Math Score)',
    labels={'Principal Component 1': 'PC1', 'Principal Component 2': 'PC2'},
    color='Math Score',
    color_continuous_scale='rainbow'
)

fig2 = px.scatter(
    pca_df,
    x='Principal Component 1',
    y='Principal Component 2',
    hover_data=['Math Score', 'Reading Score', 'Writing Score'],
    title='PCA of Student Performance (Writing Score)',
    labels={'Principal Component 1': 'PC1', 'Principal Component 2': 'PC2'},
    color='Writing Score',
    color_continuous_scale='rainbow'
)

fig3 = px.scatter(
    pca_df,
    x='Principal Component 1',
    y='Principal Component 2',
    hover_data=['Math Score', 'Reading Score', 'Writing Score'],
    title='PCA of Student Performance (Reading Score)',
    labels={'Principal Component 1': 'PC1', 'Principal Component 2': 'PC2'},
    color='Reading Score',
    color_continuous_scale='rainbow'
)

# Create subplots horizontally
fig = sp.make_subplots(rows=1, cols=3, shared_xaxes=False, shared_yaxes=False, horizontal_spacing=0.1)

# Add traces to the subplots
fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)
fig.add_trace(fig3['data'][0], row=1, col=3)

# Add labels at the top of each plot using annotations
fig.add_annotation(
    text='Math Score',
    xref='paper', yref='paper',
    x=0.07, y=1.15,
    showarrow=False,
    font=dict(size=14)
)

fig.add_annotation(
    text='Writing Score',
    xref='paper', yref='paper',
    x=0.5, y=1.15,
    showarrow=False,
    font=dict(size=14)
)

fig.add_annotation(
    text='Reading Score',
    xref='paper', yref='paper',
    x=0.9, y=1.15,
    showarrow=False,
    font=dict(size=14)
)

# Update layout for the overall figure
fig.update_layout(
    title='PCA of Student Performance',
    xaxis=dict(title='Principal Component 1 (PC1)'),
    yaxis=dict(title='Principal Component 2 (PC2)'),
    showlegend=False,
)

# Show the horizontal subplot
fig.show()


Above are three identical plots but color coded by our three main features. As you can see for the writing and reading score, the gradient of the color is nearly the same. Why? Because as we've already learned their  covariance is high.

It's different from math when compared to other features, because even if generally when writing or reading score get bigger the math score also get bigger, but because the covariance is low, we see that the gradient still from the left to right, but a little bit rotated clockwise.

One of the neat thing about PCA is that data that is related to each other will be clustered together. You can try to check above plot that:

- Students that have all scores high are clustered together
- Students that have all scores low are clustered together
- Students that have high reading and writing scores but low math scores are clustered together
- Students that have high math scores but low reading and writing scores are clustered together
- And so on

You can try to check it yourself! But hang on, what if we have a machine learning model that can help us cluster unsupervised data automatically? Hmm, I wonder if there's any machine learning model that can do that? 🤔

> Note: One term that you need to know is every features after PCA dimensionality reduction is called **principal component**. So in our case we have 2 features after reducing the dimensionality of our dataset, so we have 2 principal components.


In [ ]:
from sklearn.cluster import KMeans
import plotly.express as px

# Perform k-means clustering on the PCA-transformed data
kmeans = KMeans(n_clusters=5, random_state=0)
pca_df['Cluster'] = kmeans.fit_predict(principal_components)

# Create a scatter plot for the k-means clustering results
fig4 = px.scatter(
    pca_df,
    x='Principal Component 1',
    y='Principal Component 2',
    color='Cluster',
    title=f'K-Means Clustering on PCA Components',
    labels={'Principal Component 1': 'PC1', 'Principal Component 2': 'PC2'},
    color_continuous_scale='rainbow',
    hover_data=['Math Score', 'Reading Score', 'Writing Score'],
)

# Show the k-means clustering plot
fig4.show()


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



K-means of course! We can see above and formalize our learning about PCA will cluster the data that is related to each other together. There are so many action items that we can take from this that we might want to share to our stakeholders:

- If any students fall to the cluster number 5, 8, 17, 19, they might need major help to fix their scores (because as we can see most data that fall into those clusters are students with low scores)
- Some students on 6 and 15 got good grades on math but not on reading and writing, maybe we can help them to improve their reading and writing skills
- Some students on 4, 11, and 14 got good grades on reading and writing but not on math, maybe we can help them to improve their math skills
- Students on cluster 20 are excellent students, maybe we can give them some special treatment

So as you can see PCA is a really amazing tools that can help us to visualize especially on unsupervised data, and even we can combine it's power with other machine learning models to help us farther.

# So what's the point behind PCA?

- PCA can help us visualize hundreds of dimensions into 2 or 3 dimensions, above data use only 3 dimensions for simplicity
- Similar data will be clustered together, imagine if we're on a finance company, PCA might help us reduce hundreds of features from our customer data into 2 or 3 features, and then we might see some pattern for our customers that are likely to pay their debt, and customers that are likely to not pay their debt, some fraud customers, and so on, because we might see they cluster together
- We can use PCA as intermediate step to then feed the data into other machine learning models, such as above we can see that we can use PCA to reduce the dimensionality of our data, and then we can feed the data into K-means to cluster the data, and then we can say that some clusters are students that need help only on math, some clusters are students that need help only on reading and writing, and so on
- Other use case is we can use PCA to compress our data
- And so on

# Inference

So given we have a new student scores, for example: math score 45, reading score 80, and writing score 90, how can we manually predict the cluster number? Let's breakdown

## Standardization

Scikit by default will standardize the data for us, it's basically subtracting the mean from each data point (some standardization method also divide the data point by the standard deviation, but scikit doesn't do that by default). For the reason why this standardization is used, you can check the supplementary material. So let's check every mean of each feature:

In [ ]:
import numpy as np

mean_math = np.mean(numerical_data['math score'])
mean_reading = np.mean(numerical_data['reading score'])
mean_writing = np.mean(numerical_data['writing score'])

print("Mean Math Score:", mean_math)
print("Mean Reading Score:", mean_reading)
print("Mean Writing Score:", mean_writing)

Mean Math Score: 66.089
Mean Reading Score: 69.169
Mean Writing Score: 68.054


Now let's subtract the mean from each data point:

In [ ]:
math_score = 45
reading_score = 80
writing_score = 90

math_score_standardized = math_score - mean_math
reading_score_standardized = reading_score - mean_reading
writing_score_standardized = writing_score - mean_writing

print("Standardized Math Score:", math_score_standardized)
print("Standardized Reading Score:", reading_score_standardized)
print("Standardized Writing Score:", writing_score_standardized)

Standardized Math Score: -21.089
Standardized Reading Score: 10.831000000000003
Standardized Writing Score: 21.945999999999998


## Plotting our data

Now let's plot our data, to plot our data we need to know how every scores impact each principal component at their calculation. This impact is called **loading score** or **weight**. Let's check the loading score of our data:

In [ ]:
# Access the PCA components (weights) for each column
pca_weights = pca.components_

# Create a DataFrame to display the PCA weights
pca_weights_df = pd.DataFrame(pca_weights, columns=numerical_data.columns, index=['PC1', 'PC2'])

# Display the PCA weights
print("PCA Weights for Each Column:")
print(pca_weights_df)


PCA Weights for Each Column:
     math score  reading score  writing score
PC1   -0.562649      -0.573977      -0.594959
PC2    0.825612      -0.353292      -0.439943


The formula is simple, we just need to multiply the loading score with the standardized data point. For example for the $PC1$ our standardized data point is:

$$
\text{Standardized Math Score}: -21.089 \\
\text{Standardized Reading Score}: 10.831000000000003 \\
\text{Standardized Writing Score}: 21.945999999999998 \\
$$

With given weights:

$$
\text{Math loading score}: -0.562649\\
\text{Reading loading score}: -0.573977\\
\text{Writing loading score}: -0.594959
$$

Then we just need to multiply them:

$$
PC1 = -21.089 \times -0.562649 + 10.831000000000003 \times -0.573977 + 21.945999999999998 \times -0.594959 = -7.40801316088976
$$

This would be the value of all of our scores on the plot.

In [ ]:
pc1_calculation = pca_weights_df.loc["PC1", "math score"] * math_score_standardized + pca_weights_df.loc["PC1", "reading score"] * reading_score_standardized + pca_weights_df.loc["PC1", "writing score"] * writing_score_standardized
print("PC1 Calculation:", pc1_calculation)

pc2_calculation = pca_weights_df.loc["PC2", "math score"] * math_score_standardized + pca_weights_df.loc["PC2", "reading score"] * reading_score_standardized + pca_weights_df.loc["PC2", "writing score"] * writing_score_standardized
print("PC2 Calculation:", pc2_calculation)

PC1 Calculation: -7.408013160889752
PC2 Calculation: -30.89282344288166


So after we calculate our points, we can plot it on our PCA plot on (-7.40801316088976, -30.892823442881664):

In [ ]:
from sklearn.cluster import KMeans
import plotly.express as px

# Perform k-means clustering on the PCA-transformed data
kmeans = KMeans(n_clusters=22, random_state=0)
pca_df['Cluster'] = kmeans.fit_predict(principal_components)
data = {'Principal Component 1': [-7.40801316088976],
        'Principal Component 2': [-30.892823442881664],
        'Math Score': [45],
        'Reading Score': [80],
        'Writing Score': [90],}

pca_df_new = pd.concat([pca_df, pd.DataFrame(data)], ignore_index=True)

# Create a scatter plot for the k-means clustering results
fig4 = px.scatter(
    pca_df_new,
    x='Principal Component 1',
    y='Principal Component 2',
    color='Cluster',
    title=f'K-Means Clustering on PCA Components',
    labels={'Principal Component 1': 'PC1', 'Principal Component 2': 'PC2'},
    color_continuous_scale='rainbow',
    hover_data=['Math Score', 'Reading Score', 'Writing Score'],
)

# Show the k-means clustering plot
fig4.show()


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



As you can see above we have a new single data point plotted that hasn't been clustered yet. So let's cluster it!

## Finding the cluster

To find the cluster we need to get all centroids first.

In [ ]:
def centroids_to_dict(centroids):
    centroid_dict = {}
    for i, centroid in enumerate(centroids):
        centroid_key = f"Centroid {i+1}"
        centroid_dict[centroid_key] = centroid.tolist()
    return centroid_dict


centroid_dict = centroids_to_dict(kmeans.cluster_centers_)
print(centroid_dict)

{'Centroid 1': [15.940113122611793, -7.8049425291843795], 'Centroid 2': [-23.786263199980926, 5.244763062938197], 'Centroid 3': [70.65323917931161, 6.054481835880035], 'Centroid 4': [-6.091060746752418, -2.140101113927377], 'Centroid 5': [-36.59968608546574, -4.265391500124237], 'Centroid 6': [32.99486744215424, 8.619365303929914], 'Centroid 7': [6.704093111857857, 0.5122505600487935], 'Centroid 8': [28.812300551374967, -5.82304490390401], 'Centroid 9': [-50.03542188795403, -0.09106529160408858], 'Centroid 10': [14.182180141680067, 8.218209280648093], 'Centroid 11': [-15.293850929684183, -1.0221436700098037], 'Centroid 12': [45.29435855853409, 5.599775291786962], 'Centroid 13': [-26.85154015414675, -5.451227763385968], 'Centroid 14': [61.685551697072306, -7.267978149733127], 'Centroid 15': [-1.197215365506168, 7.955623389928318], 'Centroid 16': [23.627923896502768, 3.4588516554262885], 'Centroid 17': [1.5450331120440852, -8.769649899070018], 'Centroid 18': [-14.02760763996963, -10.2290

Then we need to calculate the distance between each centroid with our new data point and choose the smallest distance using euclidean distance:

In [ ]:
import numpy as np

def find_nearest_centroid(x, y, centroids):
    # Create a point as a NumPy array
    point = np.array([x, y])

    # Calculate the Euclidean distance between the point and all centroids
    distances = np.linalg.norm(np.array(list(centroids.values())) - point, axis=1)

    # Find the index of the nearest centroid
    nearest_centroid_index = np.argmin(distances)

    return f"cluster {nearest_centroid_index}"

nearest_centroid = find_nearest_centroid(pc1_calculation, pc2_calculation, centroid_dict)
print(f"The cluster for ({pc1_calculation}, {pc2_calculation}) is {nearest_centroid}")


The cluster for (-7.408013160889752, -30.89282344288166) is cluster 17


Now we know that our data point is closest to cluster 14.

In [ ]:
from sklearn.cluster import KMeans
import plotly.express as px

# Perform k-means clustering on the PCA-transformed data
kmeans = KMeans(n_clusters=22, random_state=0)
pca_df['Cluster'] = kmeans.fit_predict(principal_components)
data = {'Principal Component 1': [-8.488375],
        'Principal Component 2': [-30.892823442881664],
        'Math Score': [45],
        'Reading Score': [80],
        'Writing Score': [90],
        'Cluster': [14]
        }

pca_df_new = pd.concat([pca_df, pd.DataFrame(data)], ignore_index=True)

# Create a scatter plot for the k-means clustering results
fig4 = px.scatter(
    pca_df_new,
    x='Principal Component 1',
    y='Principal Component 2',
    color='Cluster',
    title=f'K-Means Clustering on PCA Components',
    labels={'Principal Component 1': 'PC1', 'Principal Component 2': 'PC2'},
    color_continuous_scale='rainbow',
    hover_data=['Math Score', 'Reading Score', 'Writing Score'],
)

# Show the k-means clustering plot
fig4.show()


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Now we know that a student with: math score 45, reading score 80, and writing score 90, is clustered on cluster 14. Basically if we check our previous notes:

- If any students fall to the cluster number 5, 8, 17, 19, they might need major help to fix their scores (because as we can see most data that fall into those clusters are students with low scores)
- Some students on 6 and 15 got good grades on math but not on reading and writing, maybe we can help them to improve their reading and writing skills
- Some students on 4, 11, and 14 got good grades on reading and writing but not on math, maybe we can help them to improve their math skills
- Students on cluster 20 are excellent students, maybe we can give them some special treatment

We can see that this student, because it's clustered on cluster 14, got really good scores on reading and writing, but not on math, so we can help them to improve their math skills.